## Analyze e2e latencies for the single_client_e2e experiment

In [ ]:
import re
import numpy as np
def get_results(path):
    computee2e = []
    consumee2e = []
    avg_batching_latencies = []
    avg_append_latency = []
    avg_batch_size = []
    total_append_per_client = []
    runtime = 0
    with open(path+"/client_node7.log", "r") as f:
        lines = f.readlines()
        for line in lines:
            if "latencies: " in line:
                consume, compute = line.strip().split('latencies: ')[-1].split(',')[-2:]
                consumee2e.append(int(consume))
                computee2e.append(int(compute))
            if "average batch size: " in line:
                avg_batch_size.append(float(line.strip().split()[-1]))
            if "average batching latency: " in line:
                avg_batching_latencies.append(float(line.strip().split()[-1]))
            if "avg latency of" in line:
                avg_append_latency.append(float(line.strip().split()[-2]))
                total_append_per_client.append(float(line.split('[single_client_e2e]:')[1].split()[3]))
            if "starting benchmark with runtime" in line:
                runtime = float(line.strip().split()[-1])

    computee2e = np.array(computee2e, dtype=int)
    consumee2e = np.array(consumee2e, dtype=int)
    avg_append_latency = np.array(avg_append_latency, dtype=float)

    print("results for computation time " + path.split("_")[-1] + " us")
    print("statistic/metric, total/compute e2e (us), consume e2e (us)")
    print(f"mean, {np.mean(computee2e):.2f}, {np.mean(consumee2e):.2f}")
    print(f"std, {np.std(computee2e):.2f}, {np.std(consumee2e):.2f}")
    print(f"p50, {np.percentile(computee2e, 50):.2f}, {np.percentile(consumee2e, 50):.2f}")
    print(f"p99, {np.percentile(computee2e, 90):.2f}, {np.percentile(consumee2e, 90):.2f}")

    print("average batch size, average batching latencies, avg throughput")
    print(f"{avg_batch_size[-1]:.2f}, {avg_batching_latencies[-1]:.2f}, {np.sum(total_append_per_client)/runtime:.2f}")

    return np.mean(computee2e), np.mean(consumee2e), np.mean(avg_append_latency)


In [ ]:
import matplotlib.pyplot as plt
total = []
consume = []
append = []


comp_times=["100", "200", "500", "800", "1000", "1200"]
for comp_time in comp_times:
    total_lat, consume_lat, append_lat = get_results(f"../results/logs/e2e_{comp_time}")
    total.append(total_lat)
    consume.append(consume_lat)
    append.append(append_lat)
    print("****************************")

fig, ax = plt.subplots()
ax.plot(comp_times, append, label='append', marker='x')
ax.plot(comp_times, consume, label='consume', marker='x')
ax.plot(comp_times, total, label='total', marker='x')

ax.set(xlabel='computation time (us)', ylabel='latency (us)', title='latency vs computation time')
ax.grid()
ax.legend()
fig.savefig("latency_vs_comp_time.png", dpi=600)
plt.show()